In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
filepath=[r"B:\project\human\human_chunk_12.csv",r"B:\project\ai\ai_chunk_12.csv","B:\project\human\human_chunk_13.csv",r"B:\project\ai\ai_chunk_11.csv","B:\project\human\human_chunk_14.csv",r"B:\project\ai\ai_chunk_10.csv","B:\project\human\human_chunk_15.csv",r"B:\project\ai\ai_chunk_9.csv"]
dfs = [pd.read_csv(filepath) for filepath in filepath]
df = pd.concat(dfs, ignore_index=True)
print(df.shape)


(120278, 2)


In [5]:
essay=df['text'].tolist()
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
# essay=df['text'].tolist()
tokens=tokenizer(essay,return_tensors='pt',padding=True,truncation=True)

In [11]:
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

# Define your model
input_length = None
input_layer = Input(shape=(input_length,))
embedding_layer = Embedding(input_dim=51000, output_dim=100, input_length=input_length)(input_layer)
conv1d_layer = Conv1D(128, 5, activation='relu')(embedding_layer)
maxpool_layer = GlobalMaxPooling1D()(conv1d_layer)
dense_layer = Dense(64, activation='relu')(maxpool_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Define the k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
accuracies = []
for train_index, test_index in kfold.split(xtrain_np):
    xtrain_cv, xtest_cv = xtrain_np[train_index], xtrain_np[test_index]
    ytrain_cv, ytest_cv = ytrain.iloc[train_index], ytrain.iloc[test_index]

    # Fit the model on the training data with validation data
    model.fit(xtrain_cv, ytrain_cv, epochs=5, batch_size=256, 
              validation_data=(xtest_cv, ytest_cv), callbacks=[early_stopping])

    # Evaluate the model on the testing data
    loss, accuracy = model.evaluate(xtest_cv, ytest_cv)
    accuracies.append(accuracy)

# Print the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print("Average accuracy: ", average_accuracy)


Epoch 1/5
301/301 [==============================] - 215s 712ms/step - loss: 0.1417 - accuracy: 0.9502 - val_loss: 0.0242 - val_accuracy: 0.9919
Epoch 2/5
301/301 [==============================] - 207s 686ms/step - loss: 0.0090 - accuracy: 0.9978 - val_loss: 0.0101 - val_accuracy: 0.9970
Epoch 3/5
301/301 [==============================] - 209s 694ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 0.0069 - val_accuracy: 0.9978
Epoch 4/5
301/301 [==============================] - 214s 711ms/step - loss: 0.0019 - accuracy: 0.9997 - val_loss: 0.0075 - val_accuracy: 0.9980
Epoch 5/5
602/602 [==============================] - 10s 17ms/step - loss: 0.0093 - accuracy: 0.9980
Epoch 1/5
301/301 [==============================] - 205s 679ms/step - loss: 0.0047 - accuracy: 0.9987 - val_loss: 6.5066e-04 - val_accuracy: 0.9999
Epoch 2/5
301/301 [==============================] - 204s 677ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 6.9992e-04 - val_accuracy: 0.9998
Epoch 3/5
301/301 [==

In [12]:
model.save("newmodel.keras")
model.save_weights('newweights.keras')

In [16]:
from tensorflow.keras.models import load_model
ml=load_model("newmodel.keras")
data=pd.read_csv(r"C:\Users\kumar\Downloads\train_drcat_01.csv")
data

,text,label,source,fold
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0
4,My argument for the development of the driverl...,0,persuade_corpus,3
...,...,...,...,...
33254,Are driverless cars really necessary? Most of ...,0,persuade_corpus,4
33255,"""Oh man I didn't make the soccer team!"", yelle...",0,persuade_corpus,1
33256,I believe that using this technology could be ...,0,persuade_corpus,0
33257,Texting & Driving\n\nUsing your phone while dr...,0,persuade_corpus,3


In [17]:
data.isnull().sum()

text      0
label     0
source    0
fold      0
dtype: int64

In [19]:
data=data.drop(["source","fold"],axis=1)

In [27]:
field=data['text'].tolist()
from transformers import LongformerTokenizer
tokenizer1=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
values=tokenizer1(field,return_tensors='pt',padding=True,truncation=True)

In [29]:
import numpy as np
tokens_numpy = values['input_ids'].detach().cpu().numpy()

In [46]:
xnew=values['input_ids'].numpy()
ynew=data['label']
xtrain_new,xtest_new,ytrain_new,ytest_new=train_test_split(x,y,test_size=0.8,random_state=42)
ynew_np=np.array(ynew)

loss, accuracy = ml.evaluate(xnew, ynew_np)
print("Loss:", loss)
print("Accuracy:", accuracy)

1040/1040 [==============================] - 47s 45ms/step - loss: 0.0051 - accuracy: 0.9989
Loss: 0.0051248399540781975
Accuracy: 0.9989476799964905
